In [2]:
import math
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from sklearn.metrics import roc_auc_score

1) Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [4]:
data = pd.read_csv(r'/Users/u17491440/PycharmProjects/enterInML/data/week_3/data-logistic.csv', names=["class", "val_1", "val_2"])
data.head()

,class,val_1,val_2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


2) Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

3) Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

4) Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

$w_1:=w_1+k\frac{1}{l}\sum_{1 \leq i \leq l}y_i x_{i1} (1-\frac{1}{1+exp(-y_i(w_1 x_{i1}+w_2 x_{i2}))})-kCw_1$

$w_2:=w_2+k\frac{1}{l}\sum_{1 \leq i \leq l}y_i x_{i2} (1-\frac{1}{1+exp(-y_i(w_1 x_{i1}+w_2 x_{i2}))})-kCw_2$

In [6]:
w_1, w_2 = 10, 10
w_1_new, w_2_new = 10, 10
k = 0.1
l = data.shape[0]
range_r = 100
num_iter = 0
C = 10

# без регуляризации
for i in tqdm(range(10000)):
    w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
    w_2 = w_2_new
    w_1 = w_1_new
    if (range_r < 10**(-5)): break

5) Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 

In [8]:
w_1, w_2 = 10, 10
w_1_new, w_2_new = 10, 10
k = 0.1
l = data.shape[0]
range_r = 100
data.iloc[0,0]
num_iter = 0
C = 10

# без регуляризации
for i in tqdm(range(10000)):
    w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
    w_2 = w_2_new
    w_1 = w_1_new
    if (range_r < 10**(-5)): break
a = [1/ (1+math.exp(-w_1*data.iloc[i, 1] - w_2*data.iloc[i,2])) for i in range(l)]
print("без регуляризации", roc_auc_score(data.iloc[:,0], a))

# С регуляризацией
for i in tqdm(range(10000)):
    w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)] - k*C*w_1_new)
    w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)] - k*C*w_2_new)
    range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
    w_2 = w_2_new
    w_1 = w_1_new
    if (range_r < 10**(-5)): break
a = [1/ (1+math.exp(-w_1*data.iloc[i, 1] - w_2*data.iloc[i,2])) for i in range(l)]
print("С регуляризацией", roc_auc_score(data.iloc[:,0], a))


без регуляризации 0.9266666666666666



С регуляризацией 0.9370476190476189


6) Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

In [10]:
w_1, w_2 = 10, 10
w_1_new, w_2_new = 10, 10
k = 0.1
l = data.shape[0]
range_r = 100
num_iter = 0
C = 10
print("Алгоритм без регуляризации")
for k in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    for i in tqdm(range(10000)):
        w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
        w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
        range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
        w_2 = w_2_new
        w_1 = w_1_new
        if (range_r < 10**(-5)): break
    a = [1/ (1+math.exp(-w_1*data.iloc[i, 1] - w_2*data.iloc[i,2])) for i in range(l)]
    print(k, " - ", roc_auc_score(data.iloc[:,0], a), " - параметр сходимости ", range_r, " - число этераций ", i)   

Алгоритм без регуляризации


0.05  -  0.9265714285714286  - параметр сходимости  9.936533399073441e-06  - число этераций  994


0.1  -  0.9266666666666666  - параметр сходимости  9.798284446237108e-06  - число этераций  29


0.2  -  0.9266666666666666  - параметр сходимости  9.69694775095824e-06  - число этераций  14


0.3  -  0.9267619047619047  - параметр сходимости  9.584063740644553e-06  - число этераций  5


0.4  -  0.9267619047619047  - параметр сходимости  9.727245757720137e-06  - число этераций  2


0.5  -  0.9267619047619047  - параметр сходимости  9.702918209419873e-06  - число этераций  1


0.6  -  0.9267619047619047  - параметр сходимости  8.805134509989033e-06  - число этераций  1


0.7  -  0.9267619047619047  - параметр сходимости  8.812373136124114e-06  - число этераций  0


0.8  -  0.9267619047619047  - параметр сходимости  8.401012097789157e-06  - число этераций  0


7) Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

Начальное приближение влияет слабо, в основном на число этераций

$w_1, w_2, w_{1,new}, w_{2, new} = 10, 10, 10, 10$ без регуляризации: 0.927 (522); с регуляризацией 0.937 (54)     
$w_1, w_2, w_{1,new}, w_{2, new} = 0, 0, 0, 0$ без регуляризации: 0.927 (243); с регуляризацией 0.937(54)       
$w_1, w_2, w_{1,new}, w_{2, new} = 10, 0, 10, 0$ без регуляризации: 0.927 (595); с регуляризацией 0.937 (52)    
$w_1, w_2, w_{1,new}, w_{2, new} = 0, 10, 0, 10$ без регуляризации: 0.927 (537); с регуляризацией 0.937 (48)   
$w_1, w_2, w_{1,new}, w_{2, new} = 10, 10, 0, 0$ без регуляризации: 0.927 (240); с регуляризацией 0.937 (48)  
$w_1, w_2, w_{1,new}, w_{2, new} = 0, 0, 10, 10$ без регуляризации: 0.927 (527); с регуляризацией 0.937 (54)  
 


In [20]:
w_1, w_2 = 0, 0
w_1_new, w_2_new = 10, 10
k = 0.1
l = data.shape[0]
range_r = 100
num_iter = 0
C = 10

# без регуляризации
for i in tqdm(range(10000)):
    w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)])
    range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
    w_2 = w_2_new
    w_1 = w_1_new
    if (range_r < 10**(-5)): break
a = [1/ (1+math.exp(-w_1*data.iloc[i, 1] - w_2*data.iloc[i,2])) for i in range(l)]
print("без регуляризации", roc_auc_score(data.iloc[:,0], a))

# С регуляризацией
for i in tqdm(range(10000)):
    w_1_new = w_1_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,1]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)] - k*C*w_1_new)
    w_2_new = w_2_new + k*1/l*sum([ data.iloc[i,0]*data.iloc[i,2]*(1 - 1/(1 + math.exp(-data.iloc[i,0]*(w_1*data.iloc[i,1] + w_2*data.iloc[i,2]))))  for i in range(l)] - k*C*w_2_new)
    range_r = ((w_1_new - w_1)**2 + (w_2_new - w_2)**2)**(1/2)
    w_2 = w_2_new
    w_1 = w_1_new
    if (range_r < 10**(-5)): break
a = [1/ (1+math.exp(-w_1*data.iloc[i, 1] - w_2*data.iloc[i,2])) for i in range(l)]
print("С регуляризацией", roc_auc_score(data.iloc[:,0], a))

без регуляризации 0.9266666666666666


С регуляризацией 0.9370476190476189


Какой AUC-ROC получается у классификаторов на обучающей выборке без регуляризации и при ее использовании (укажите два числа через пробел)?
0.926 0.937